# Custom model implementation and validation

## Introduction

#### **Custom model training**

We will be using [Scikit-learn](https://scikit-learn.org/stable/) python library API to use our model so that it can work like a standard scikit-learn model.

That's convenient, because Scikit-learn provides:
- feature preprocessing trasforms (e.g. Onehot encoding, label encoding, feauture scaling, etc);
- most of the core algorithms (Random Forests, Logistic/Linear regression, GBMs, etc);
- metrics and other convenience functions for tracking model performance.


#### **Machine learning vocab**

**Function** $f(\vec{x})$ which is approximating our (experimental) data is often called a **"model"**,

The **parameters** of a function may be interchangeably used with the term **"weights"** (of a model),

**Indepenent variables** (input) $x_1, x_2, ..., x_n$ of a function (model) are often called **"features"**, and by default vectorized into n-dementional space as a vector $\vec{x} = (x_1, x_2, ..., x_n)$,

**Dependent variable** (output) $y$ of a function (model) is sometimes called **"target"**, or **"label"**, or **"ground truth"**, especially, if it's derived from experiments.

**Predictions** are simply the values calculated from our function (model) $\hat{y} = f(\vec{x})$

#### **Feature selection**

Feature selection is a problem-specific task (in most general case classification or regression), 
we will be using two approaches for regression models feature selection:
- F-statistics
- cross-validated scores (less optimal)

Later we may introduce other techniques


**Dimentionality reduction**

We will implenet PCA and t-NSE templates with visualization on a publicly available dataset

## Technical detail


### Prep: importing libraries

In [8]:
import pandas as pd
import numpy as np


#import matplotlib.pyplot as plt # uncomment if you prefer matplotlib
import plotly.graph_objects as go # use this if you prefer interactive plotly graphs
import plotly.express as px # also interactive plots, allows more compact notation but more demanding on the input data format
from plotly.subplots import make_subplots # to plot several plots in one figure

from scipy.optimize import curve_fit #for fitting a curve to data
from scipy.optimize import minimize
from scipy.optimize import basinhopping, brute

import statsmodels.api as sm

from sklearn import datasets #to show an example
#from sklearn.datasets import make_regression # to show an example of feature selection

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures # use if you want to add polynomial features
#from sklearn.preprocessing import OneHotEncoder # uncomment this use if if you have binary features and want to use them in your model

from sklearn.model_selection import train_test_split

from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression, SelectFromModel
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_val_predict, cross_val_score, KFold

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE   

import latexify



## Prep: Data

### Data import

Import your data from an excel file or csv, indicating the path

In [ ]:
'''
# Read the Excel file with experiment values into a dataframe (df)
df = pd.read_excel('path/to/excel/file.xlsx', sheet_name='experiment') # in case of xls file
# df=pd.read_csv('experiment.csv') # in case of csv file
'''

### Data split

We need to split the data into:
- trian set - on which we train our model
- test set - on which we extimate the performance of our model

In [ ]:
''''
# Isolate feature vector
X = df.drop('y', axis=1)

# Isolate target
y = df['y']

# Split the data into train, test (and in special cases another validation) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # First we take 80% for training and 20% set apart in test set

# Have a look at the shapes of the datasets
print("Train set shape:", X_train.shape)
print("Test set shape:", X_test.shape)
'''


### Data preprocessing

Skip for now

Feature scaling is used in case of distance based algorithms (e.g. k-NN, SVM, PCA, LDA) and in case of gradient descent based algorithms (e.g. linear regression, logistic regression, neural networks)

It is not necessary for decision trees, random forests, naive bayes, k-means, apriori, eclat, FP-growth, etc.

It is also not necessary for linear regression if we use the normal equation to find the weights

It is also not necessary for logistic regression if we use the maximum likelihood estimation to find the weights


Log or third root trasform is applied in case of a skewed distribution of the target variable, but then predictions should be transformed back.

Other preprocessing steps can be applied here, such as standardization, normalization, adding polynomial features, as well as feature selection at this point


In [ ]:
''''
sc      = StandardScaler() # Standardize features by centralizing mean to 0 the mean and scaling to unit variance
X_train = sc.fit_transform(X_train) # fit and transform the training set
X_test  = sc.transform(X_test) # transform the test set by the SAME parameters as the TRAINING set
'''

## Model Encoding

### Regression model

Template is the following


´´´
    
    class Name_of_Custom_Regressor(BaseEstimator, RegressorMixin):
        
        def __init__(self, param1=1, param2=2):
            self.param1 = param1
            self.param2 = param2
        
        # Fit the model to the training data
        def fit(self, X, y=None):
            pass
            
        # Use the fitted model to make predictions on new data
        def predict(self, X, y=None):
            pass
        
        # Calculate the model's performance on the given data and labels
        def score(self, X, y):
        pass
´´´

#### Example on an exponential decay curve

Assume our custom model is exponential decay function
$$ f(x) = a e^{-kx}+b $$

In [9]:
# Let's define a custom estimator class
class Name_of_Custom_Regressor(BaseEstimator, RegressorMixin):
    def __init__(self, starting_values=[1.,1.e-5,1.], **kwargs,): # Initial guess for a faster convergiance of the optimization
        self.starting_values = starting_values
        self.kwargs = kwargs
        self.params = None
    
    def fit(self, X, y=None):
            self.params, _ = curve_fit(self.exp_decay_f, X, y, p0=self.starting_values)
            self.coef_ = self.params 
    
    def predict(self, X, y=None):
        return self.exp_decay_f(X, *self.params)
    
    def score(self, X, y):
        # Calculate the model's performance on the given data and labels
        cv_score = cross_val_score(self, X, y, cv=3, scoring='neg_mean_squared_error')        
        # If not defiened, it's by default r-squared error https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score

        r2 = r2_score(y, self.predict(X))
        return cv_score.mean(), r2

    @staticmethod
    def exp_decay_f(X, a, k, b):
        return a * np.exp(-k*X) + b  # Exponential decay function   

Here is another usefull [Quantile Regression example](https://github.com/conormm/blog/blob/master/_notebooks/2021-04-11-lesser-known-data-scienec-techniques.ipynb)

### Classification model (to be added later if needed)

## Model usage

Assume we have a ground truth model with the following parameters (which we don't know and would like to derive from our experiments)

$$ f(x) = 2 e^{-0.5x}+1 $$

We performed an experiment, and obtained some dataset with some noise

In [10]:
# A ground truth model with the following parameters
X = np.linspace(0, 10, 100) # Generate 100 evenly spaced numbers between 0 and 10
a = 2
k = 0.5
b = 1
Y = a * np.exp(-k*X) + b

# We performed an experiment, and obtained some dataset with some noise (adding gaussian noise to the ground truth data)
noise = np.random.normal(0, 0.1, len(Y))
Y_with_noise = Y + noise

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_with_noise, test_size=0.2, random_state=42)


In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=X, y=Y_with_noise, mode='markers'))

fig.update_layout(title='Noisy Experiment Data before splitting into train and test set and before fitting the model',
                  xaxis_title='X',
                  yaxis_title='Y',
                  width=900)

fig.show()

Once we have implemented the class, we can create an instance of the estimator and use it like any other scikit-learn estimator with fit, predict, score funtions

In [12]:
model = Name_of_Custom_Regressor()
model.fit(X_train, Y_train) # training our model on training dataset to obtain parameters (weights) of the model
predictions = model.predict(X_test) # making predictions on the test dataset to evaluate the model performance
score = model.score(X_test, Y_test) # evaluating the model performance on the test dataset

Extract the parameters of the trained model and get some datapoints for visualisation

In [13]:
a_fit = model.params[0]
k_fit = model.params[1]
b_fit = model.params[2]
X_fit = np.linspace(0, 10, 100)
Y_fit = model.predict(X_fit)

In [14]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=X_test, 
                         y=Y_test, 
                         mode='markers', 
                         name='Test Data',
                         marker=dict(color='rgb(93, 105, 177)')))
                                     
fig.add_trace(go.Scatter(x=X_train, 
                         y=Y_train, 
                         mode='markers', 
                         name='Train Data', 
                         marker=dict(color='rgb(229, 134, 6)')))

fig.add_trace(go.Scatter(x=X_fit, y=Y_fit, mode='lines', name='Fitted Curve'))
fig.add_trace(go.Scatter(x=X_fit, y=Y, mode='lines', name='Ground Truth Curve', line=dict(color='gray', dash='dash')))

fig.update_layout(title='Custom model (exponential decay function) fitted to experimental data on the train set and validated on the test set',
                  xaxis_title='X',
                  yaxis_title='Y',
                  width=1100,
                  annotations=[dict(x=5, y=2, text="Cross-validated MSE: {:.4f}".format(score[0]), showarrow=False, font=dict(color='rgb(93, 105, 177)')),
                               dict(x=5, y=1.5, text="R2 Score: {:.4f}".format(score[1]), showarrow=False, font=dict(color='rgb(93, 105, 177)')),
                               dict(x=5, y=2.5, text="Fitted Parameters: a={:.2f}, k={:.2f}, b={:.2f}".format(a_fit, k_fit, b_fit), showarrow=False, font=dict(color='rgb(229, 134, 6)')),
                               dict(x=5, y=3, text="Ground truth Parameters: a={:.2f}, k={:.2f}, b={:.2f}".format(a, k, b), showarrow=False)])
fig.show()

It is recommended to also implement additional methods such as get_params and  set_params to make your estimator compatible with scikit-learn's hyperparameter tuning utilities. 
More information and examples of custom regressors in [the scikit-learn documentation](https://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator)


### Any other functions are welcome

In [15]:
# This is the model function, e.g., an exponential decay
def model(x, a, b):
    return a * np.exp(-b * x)

#shifted exponential decay
def exp_func(x, a, b):
    return a * np.exp(-b * x)

#shifted squared exponential decay
def exp_squared_func(x, a, b, c):
    return a * np.exp(-b * x*x) + c

#linear function
def linear_func(x, a, b):
    return a*x + b


Have a look at the functions in a more convenient way

In [16]:
@latexify.function
def model_show(x, a, b):
    return a * e**(-b * x)

model_show # Displays the expression.

In [17]:
@latexify.function
def exp_func_show(x, a, b, c):
    return a * e**(-b * x) + c

exp_func_show # Displays the expression.

In [18]:
@latexify.function
def exp_squared_func_show(x, a, b, c):
    return a * e**(-b * x**2) + c

exp_squared_func_show


In [19]:
@latexify.function

def linear_func_show(x, a, b):
    return a*x + b

linear_func_show

### Polynomial function


In [51]:
coefficients = np.polynomial.polynomial.Polynomial.fit(X_train, Y_train, 3)


print(coefficients.coef[::-1]) # The coefficients are in the reverse order, so we reverse them to get the correct order


[-0.50821901  0.74475361 -0.4310122   1.13218467]


In [57]:

# fitted values
X_fit_poly = np.linspace(0, 10, 100)
Y_fit_poly  = np.polyval(coefficients.coef[::-1], X_fit_poly)


# the fitted curve
fig = go.Figure()

fig.add_trace(go.Scatter(x=X_fit_poly, y=Y_fit_poly, mode='lines'))

# the experimental data
fig.add_trace(go.Scatter(x=X_test, 
                         y=Y_test, 
                         mode='markers', 
                         name='Test Data',
                         marker=dict(color='rgb(93, 105, 177)')))
                                     
fig.add_trace(go.Scatter(x=X_train, 
                         y=Y_train, 
                         mode='markers', 
                         name='Train Data', 
                         marker=dict(color='rgb(229, 134, 6)')))


# Set the axis labels and title
fig.update_layout(
    xaxis_title='x',
    yaxis_title='y',
    title='Fitted Polynomial'
)

# Show the plot
fig.show()


### Example on real world data (regression)

[Diabetes dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes)

Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline.

Features:
- age age in years
- sex
- bmi body mass index
- bp average blood pressure
- s1 tc, total serum cholesterol
- s2 ldl, low-density lipoproteins
- s3 hdl, high-density lipoproteins
- s4 tch, total cholesterol / HDL
- s5 ltg, possibly log of serum triglycerides level
- s6 glu, blood sugar level


Target:
Column 11 is a quantitative measure of disease progression one year after baseline

Original [source](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)

In [26]:
diab = datasets.load_diabetes()

X_diab = diab.data
y_diab = diab.target

W_diab = np.zeros(X_diab.shape[1]) #zero weights for each feature

Split the dataset into training and test sets

In [27]:
X_train_diab, X_test_diab, y_train_diab, y_test_diab = train_test_split(X_diab, y_diab, test_size=0.2, random_state=1234)

It's a good practice is to normalize the feature vector for a linear regression
$$ x' = \frac{x - \mu}{\sigma} $$

**NB:** $\mu$ and $\sigma$ are calculated from the train set and aplied to the test set


In [28]:
sc      = StandardScaler() # Standardize features by centralizing mesn to 0 the mean and scaling to unit variance
X_train_diab = sc.fit_transform(X_train_diab) # fit and transform the training set
X_test_diab  = sc.transform(X_test_diab) # transform the test set by the SAME parameters as the TRAINING set

## Features' prediction power estimation

We may select features that explain the most variance in our dependant vaiable

How to use our encoded exponential function on X, Y dataset

In [29]:
model = Name_of_Custom_Regressor() 
model.fit(X_train, Y_train) # training our model on training dataset to obtain parameters (weights) of the model
predictions = model.predict(X_test) # making predictions on the test dataset to evaluate the model performance
score = model.score(X_test, Y_test) # evaluating the model performance on the test dataset, returns the cross-validated MSE and R2 score

We encoded our model performance metrix to be cross-validated MSE and R2 score. Here are the values of our model for exponential function

In [30]:
print(f'Cross-validated MSE: {score[0]:.4f}')
print(f'R2 score: {score[1]:.4f}')

Cross-validated MSE: -0.0128
R2 score: 0.9663


But what's the predictive power of the parameters of the model

In [31]:
# Instantiate the feature_selection_method object
feature_selection_method = SelectKBest(score_func=f_regression, k=10)

# Get the scores of the features
feature_selection_method.fit(X_test_diab, y_test_diab)

# Extract the names of the features from the initial dataset
feature_names = diab.feature_names

# Get the scores of the features
feature_scores = feature_selection_method.scores_

# Sort the features and scores in descending order
sorted_indices = sorted(range(len(feature_scores)), key=lambda k: feature_scores[k], reverse=True)
sorted_feature_names = [feature_names[i] for i in sorted_indices]
sorted_feature_scores = [feature_scores[i] for i in sorted_indices]

# legend with feature explanations
feature_explanations = {
    'age': 'age in years',
    'sex': 'sex',
    'bmi': 'body mass index',
    'bp': 'average blood pressure',
    's1': 'tc, total serum cholesterol',
    's2': 'ldl, low-density lipoproteins',
    's3': 'hdl, high-density lipoproteins',
    's4': 'tch, total cholesterol / HDL',
    's5': 'ltg, serum triglycerides level',
    's6': 'glu, blood sugar level'
}


In [32]:
fig = go.Figure(
    data=[go.Bar(x=sorted_feature_names, y=sorted_feature_scores)],
    layout=dict(
        title='Feature Scores', 
        xaxis_title='Features', 
        yaxis_title='Score, F-value',
    )
)

for i, feature_name in enumerate(sorted_feature_names):
    fig.add_annotation(
        x=feature_name,
        y=sorted_feature_scores[i] + 10,
        text=feature_explanations.get(feature_name, 'Unknown'),
        textangle = -45,
        showarrow=False,
        font=dict(size=10)
    )

fig.show()

Feature inportance estimation with kfold cross validation.

For that we should train and evaluate our model on different subsets of our trainin gdata and calculate the feature importance for each subset.

The importance of the feature in this case is the avarage absolute value of its corresponding weight over all splits. For linear regression the weights are 


Example on our custom exp decay model

NB: in case of error doublecheck the X dataset (might appear as global variable several times)

In [33]:
# call KFold object from 
kf = KFold(n_splits=5, shuffle=True, random_state=1) # 5 splits, shuffle the data, and set the random state for reproducibility

# Our custom model (exponential decay in the exanple above)
model = Name_of_Custom_Regressor() 

# An empty list to hold the feature importances
feature_importances = []

# Iteration over each split
for train_index, test_index in kf.split(X):
    # Split the data
    X_train_cv, X_test_cv = X[train_index], X[test_index]
    Y_train_cv, Y_test_cv = Y_with_noise[train_index], Y_with_noise[test_index]

    # Fit the model
    model.fit(X_train_cv, Y_train_cv)

    # Calculate the feature importances and append to the list
    feature_importances.append(np.abs(model.coef_))

# Average the feature importances over all splits
average_importance = np.mean(feature_importances, axis=0)


# Print the feature importances
sorted_indices = np.argsort(average_importance)[::-1]  # Sort the indices in descending order
for i in sorted_indices:
    print(f'Feature {i}: {average_importance[i]:.3f}')

Feature 0: 2.042
Feature 2: 0.977
Feature 1: 0.486


### Why correlation doesn't help

In [34]:
# Get the number of features
num_features = X_diab.shape[1]

# Create a figure with subplots
fig = make_subplots(rows=num_features, cols=1)

# Iterate over each feature
for i in range(num_features):
    # Scatter plot of the feature with y_diab
    fig.add_trace(go.Scatter(x=X_diab[:, i], y=y_diab, mode='markers'), row=i+1, col=1)

    # Set x and y axis labels
    fig.update_xaxes(title_text=f'Feature {feature_names[i]}', row=i+1, col=1)
    fig.update_yaxes(title_text='Diab progression', row=i+1, col=1)

# Update layout
fig.update_layout(height=600*num_features, showlegend=False)

# Show the plot
fig.show()


Let's define a function creating pairs of our variables

In [ ]:
from itertools import combinations

def get_pairs(X):
    num_features = X.shape[1]
    pairs = list(combinations(range(num_features), 2))
    return num_features, pairs

Run the function of our features

In [ ]:
num_features, pairs = get_pairs(X_diab)
print("Number of features:", num_features)
print("Pairs of features:", pairs)
print("Number of pairs:", len(pairs))


In [ ]:
fig = make_subplots(rows=len(pairs), cols=1)

# Plot pairwise scatter plots
for i, (feature1, feature2) in enumerate(pairs):
    fig.add_trace(
         go.Scatter(
              x=X_diab[:, feature1],
              y=X_diab[:, feature2],
              mode='markers',
              marker=dict(color='#C00000')
              ),
              col=1,
              row=i+1
            )
    fig.update_xaxes(title_text=f'Feature {feature_names[feature1]}', row=i+1, col=1)
    fig.update_yaxes(title_text=f'Feature {feature_names[feature2]}', row=i+1, col=1)

# Update layout
fig.update_layout(height=len(pairs)*300, showlegend=False)

# Show the plot
fig.show()


## Dimentionality reduction template

Let's implement PCA decomposition to compress our data into a 3-dimensional subspace so we can visualize it as a scatter plot.
Showing it on diab dataset example

In [35]:
# initializing t-SNE dimensionality reduction and implementing it
pca_3D = PCA(n_components = 3).fit(X_diab)
X_pca = pca_3D.transform(X_diab)

#
df_pca_3D = pd.DataFrame(X_pca,columns = ['principal_component_1','principal_component_2','principal_component_3'])
df_pca_3D['Diabetes Progression'] = y_diab

In [36]:
df_pca_3D.head()

,principal_component_1,principal_component_2,principal_component_3,Diabetes Progression
0,0.027930,-0.092601,0.028026,151.0
1,-0.134687,0.065263,0.001328,75.0
2,0.012944,-0.077764,0.035162,141.0
3,0.002344,0.018183,-0.095750,206.0
4,-0.035979,0.038621,-0.002723,135.0


We preserved around 67% of the variance with PCA

In [37]:
# pca.explained_variance_ration_ returns a list where it shows the amount of variance explained by each principal component.
sum(pca_3D.explained_variance_ratio_)

0.6722496686876479

In [38]:
fig_pca = px.scatter_3d(
    df_pca_3D, 
    x = 'principal_component_1', 
    y = 'principal_component_2', 
    z = 'principal_component_3',
    color='Diabetes Progression')
fig_pca.update_layout(title="3D PCA Clustering attempt of Diabetes Dataset")

fig_pca.show()

### t-SNE dimensionality reduction

In [39]:
# initializing t-SNE dimensionality reduction implementation
tsne = TSNE(n_components=3, random_state=42) #make instance of t-SNE
X_tsne = tsne.fit_transform(X_diab)

# dataframe with the t-SNE results
df_tsne = pd.DataFrame(X_tsne, columns=['Dimension 1', 'Dimension 2', 'Dimension 3'])
df_tsne['Diabetes Progression'] = y_diab

# 3D plot
fig_tsne = px.scatter_3d(df_tsne, 
                    x='Dimension 1', 
                    y='Dimension 2', 
                    z='Dimension 3', 
                    color='Diabetes Progression')
fig_tsne.update_layout(title="3D t-SNE Clustering attempt of Diabetes Dataset")
fig_tsne.show()


### Save the plots

In [ ]:
import os
import plotly.offline as offline


results_folder = '/Users/vschastlivaia/Library/CloudStorage/OneDrive-IBEC/Documents/GitHub/model-implementation-and-validation/Results'

# the line below will create the 'Results' folder if it doesn't exist
# os.makedirs(results_folder, exist_ok=True) 

# Save the plots in the 'Results' folder
offline.plot(fig_pca, filename=os.path.join(results_folder, 'pca_plot.html'), auto_open=False)
offline.plot(fig_tsne, filename=os.path.join(results_folder, 'tsne_plot.html'), auto_open=False)

To save as static png images uncomment the code below

In [ ]:
#import kaleido
#import plotly.io as pio
#
#fig_pca.write_image(os.path.join(results_folder, 'pca_plot.png'))
#fig_tsne.write_image(os.path.join(results_folder, 'tsne_plot.png'))

### Chosing a color palette

In [23]:
px.colors.qualitative.swatches()

In [25]:
px.colors.qualitative.Prism

['rgb(95, 70, 144)',
 'rgb(29, 105, 150)',
 'rgb(56, 166, 165)',
 'rgb(15, 133, 84)',
 'rgb(115, 175, 72)',
 'rgb(237, 173, 8)',
 'rgb(225, 124, 5)',
 'rgb(204, 80, 62)',
 'rgb(148, 52, 110)',
 'rgb(111, 64, 112)',
 'rgb(102, 102, 102)']

# References

1. [Scikit-learn documentation](https://scikit-learn.org/stable/index.html)
2. [Plotly documentation](https://plotly.com/python/)
3. Great ideas from Conor's [GitHub](https://github.com/conormm) and [blog](https://conormm.github.io/blog/)
4. Multiple publications at [Towardsdatascience](https://towardsdatascience.com/)
5. Foundations from Deeplearning.AI [cources](https://www.deeplearning.ai/courses/machine-learning-specialization/) by Andrew Ng
6. My previous projects for NARMYD and IDIBAPS under NDA untill they publish their results